In [1]:
from dataclasses import dataclass, field
import sys, os
sys.path.insert(1,os.path.abspath('../src'))

Import our code under test.

In [2]:
from fpipeline import pipeline, stepfn, conditionfn, variables, if_, not_, and_, or_, Variable, Attribute, VariableContext

Load our Test framework
    

In [3]:
from tester import Test, set_test_reporter

from notebook_reporter import NotebookTestReporter

REPORTER = NotebookTestReporter()

set_test_reporter(REPORTER)

Test our Test framework

In [4]:
Test("Test Test", None).equals(None)

✅ Test Test: OK

In [5]:
Test("Test Negate Test", None).negate.equals(False)

✅ Test Negate Test: OK

#### Test Data Setup

Our test setup consists of a context class `ctx`, a tracing step function `tstep`, and a predicte `has_value`

Executing a `tstep` adds the arguments to a list in `ctx.trace`. This allows us to verify that the step was called, and that the arguments were properly passed.

The `has_value` condition function compares the valuie in `ctx.value`, returning `True` if it equals
the supplied value.

The `get_trace` function returns the `ctx.trace` list; it can be used in `.apply` to extract it for comparison.

The `ok` step function just returns `"OK"`.

The `fail` step function just returns `"Failed"`

The `true` condition function returns `True`.

The `false` condition function returns `False`.

In [6]:
@dataclass
class ctx:
    value: any = None
    trace: list[tuple] = field(default_factory=list)

In [7]:
@stepfn
def tstep(data: ctx, *args, **kwargs):
    d = (*args, {**kwargs})
    data.trace.append(d)
    return data

def get_trace(data: ctx):
    return data.trace

@stepfn
def ok(_: ctx):
    return 'OK'

@stepfn
def fail(_: ctx):
    return "Failed"

In [8]:
@conditionfn
def has_value(data: ctx, value):
    return data.value == value

@conditionfn
def true(_: ctx):
    return True

@conditionfn
def false(_: ctx):
    return False

#### Test Cases

In [9]:
Test("@stepfn step takes a single argument",
    tstep(7, 8).__code__.co_argcount
).equals(1)

✅ @stepfn step takes a single argument: OK

In [10]:
Test("@conditionfn condition takes a single argument",
    has_value(7, 8).__code__.co_argcount
).equals(1)

✅ @conditionfn condition takes a single argument: OK

In [11]:
Test("@stepfn capture args",
    tstep(7, 5)(ctx()).trace
).equals([(7, 5, {})])

✅ @stepfn capture args: OK

In [12]:
Test("@conditionfn capture args (true)",
    has_value(7)
).apply(ctx(7)).is_true()

✅ @conditionfn capture args (true): OK

In [13]:
Test("@conditionfn capture args (false)",
    has_value(7)
).apply(ctx()).is_false()

✅ @conditionfn capture args (false): OK

In [14]:
Test("Simple pipeline",
    pipeline(
        tstep(7),
        tstep(78)
        )
 ).apply(ctx()).call(get_trace).equals([
    (7, {}),
    (78, {})
    ])

✅ Simple pipeline: OK

#### Conditional Tests

In [15]:
Test("if_ True",
    if_(
        true(),
        ok()
        )
).apply(ctx()).equals('OK')

✅ if_ True: OK

In [16]:
Test("if_ False",
     if_(
        false(),
        ok()
     )
).apply(ctx()).equals(None)


✅ if_ False: OK

In [17]:
Test("if_ True else",
    if_(
        true(),
        ok(),
        fail()
    )
).apply(ctx(42)).equals('OK')

✅ if_ True else: OK

In [18]:
Test("if_ False else",
     if_(
         false(),
         fail(),
         ok()
     )
     ).apply(ctx()).equals('OK')


✅ if_ False else: OK

In [19]:
Test("not_ True",
    not_(true())
).apply(ctx()).equals(False)

✅ not_ True: OK

In [20]:
Test("not_ False",
     not_(false())
     ).apply(ctx()).equals(True)

✅ not_ False: OK

In [21]:
Test("or_ 0",
or_()
).apply(ctx()).equals(False)

✅ or_ 0: OK

In [22]:
Test("or_ T",
     or_(true())
     ).apply(ctx()).equals(True)


✅ or_ T: OK

In [23]:
Test("or_ F",
or_(false())
).apply(ctx()).equals(False)

✅ or_ F: OK

In [24]:
Test("or_ F T",
     or_(false(), true())
     ).apply(ctx()).equals(True)

✅ or_ F T: OK

In [25]:
Test("or_ T F",
     or_(true(), false())
     ).apply(ctx()).equals(True)

✅ or_ T F: OK

In [26]:
Test("or_ T T",
     or_(true(), true())
     ).apply(ctx()).equals(True)

✅ or_ T T: OK

In [27]:
Test("and_ 0",
    and_()
).apply(ctx()).equals(True)

✅ and_ 0: OK

In [28]:
Test("and_ T",
     and_(true())
     ).apply(ctx()).equals(True)

✅ and_ T: OK

In [29]:
Test("and_ F",
     and_(false())
     ).apply(ctx()).equals(False)

✅ and_ F: OK

In [30]:
Test("and_ T F",
    and_(true(), false())
    ).apply(ctx()).equals(False)

✅ and_ T F: OK

In [31]:
Test("and_ F T",
     and_(false(), true())
     ).apply(ctx()).equals(False)

✅ and_ F T: OK

In [32]:
Test("and_ T T",
    and_(true(), true())
    ).apply(ctx()).equals(True)

✅ and_ T T: OK

In [33]:
Test("and_ F F",
     and_(false(), false())
     ).apply(ctx()).equals(False)

✅ and_ F F: OK

#### Variable Tests

In [34]:
@stepfn
def f0(data: ctx):
    with variables(ctx) as vars:
        return vars
Test('Close vars',
    f0()
).apply(ctx()).call(lambda c: c.closed).is_true()

✅ Close vars: OK

In [35]:
@stepfn
def f1(data: ctx):
    with variables(ctx) as vars:
        #v1, v2 = vars.variables('v1', 'v2')
        #a1, a2 = vars.attributs('value')
        return vars
Test('Clear vars',
     f1()
     ).apply(ctx()).call(lambda c: len(c.variables)).equals(0)

✅ Clear vars: OK

In [36]:
@stepfn
def f2(data: ctx):
    c = VariableContext(data)
    v = c.variable('v')
    a = c.attribute('a')
    c.close()
Test("Check vars lifecycle",
    f2()
    ).apply(ctx()).equals(None)

✅ Check vars lifecycle: OK

In [37]:
@stepfn
def f3(data: ctx):
    with variables(data) as vars:
        v1, v2 = vars.variable('v1', 'v2')
        a1, a2 = vars.attribute('a1', 'a2')
        return list(vars.variables.keys())

Test('Check vars recorded',
     f3()
     ).apply(ctx()).equals(['v1', 'v2', 'a1', 'a2'])


✅ Check vars recorded: OK

In [38]:
@stepfn
def f4(data: ctx):
    with variables(data) as vars:
        (a) = vars.attribute('value')
        return a.value
Test("Attribute.value",
    f4()
).apply(ctx(55)).equals(55)

✅ Attribute.value: OK

### Test Summary

In [39]:
REPORTER.report()

✅ Test Test: OK ✅ Test Negate Test: OK ✅ @stepfn step takes a single argument: OK ✅ @conditionfn condition takes a single argument: OK ✅ @stepfn capture args: OK ✅ @conditionfn capture args (true): OK ✅ @conditionfn capture args (false): OK ✅ Simple pipeline: OK ✅ if_ True: OK ✅ if_ False: OK ✅ if_ True else: OK ✅ if_ False else: OK ✅ not_ True: OK ✅ not_ False: OK ✅ or_ 0: OK ✅ or_ T: OK ✅ or_ F: OK ✅ or_ F T: OK ✅ or_ T F: OK ✅ or_ T T: OK ✅ and_ 0: OK ✅ and_ T: OK ✅ and_ F: OK ✅ and_ T F: OK ✅ and_ F T: OK ✅ and_ T T: OK ✅ and_ F F: OK ✅ Close vars: OK ✅ Clear vars: OK ✅ Check vars lifecycle: OK ✅ Check vars recorded: OK ✅ Attribute.value: OK 32 successes, 0 failures, 0 errors